In [0]:
import os

In [0]:
#############################
#READ ALL NIGHT STAY FROM UAE
UAE_RESERVATION_STAT_DAILY = (spark.read
    .format('com.databricks.spark.csv')
    .options(delimiter="^", header=True,nanValue="",multiLine=True,quote="\"")
    .load('dbfs:/mnt/jumseg/reservation/UAE_RESERVATION_STAT_DAILY.csv'))

In [0]:
#SELECT ONLY REQUIRED VARIABLES
UAE_RESERVATION_STAT_DAILY_PROCESSED = UAE_RESERVATION_STAT_DAILY.select('RESORT','SOURCE_CODE','MARKET_MAIN_GROUP','MARKET_CODE','CHANNEL','WALKIN_YN','INSERT_DATE','BUSINESS_DATE','PSUEDO_ROOM_YN','ROOM_CLASS','ROOM_CATEGORY','RESV_NAME_ID','RESV_STATUS','CANCELLATION_DATE','TRUNC_BEGIN_DATE','TRUNC_END_DATE','NIGHTS','ROOM_REVENUE','FOOD_REVENUE','TOTAL_REVENUE','PACKAGE_ROOM_REVENUE','PACKAGE_FOOD_REVENUE','NAME_ID','ADULTS','CHILDREN','VIP_STATUS','BIRTH_DATE','MEMBERSHIP_ID','COUNTRY','REGION_CODE','NATIONALITY')

In [0]:
#SAVE INTO A SMALLER FILE
UAE_RESERVATION_STAT_DAILY_PROCESSED.coalesce(1).write.format("com.databricks.spark.csv").options(header=True).save("dbfs:/mnt/jumseg/reservation/UAE_RESERVATION_STAT_DAILY_PROCESSED.csv")

In [0]:
#############################
#READ SMALLER FILE AS PANDAS
import pandas as pd
UAE_RESERVATION_STAT_DAILY_PROCESSED_pd = pd.read_csv('/dbfs/mnt/jumseg/reservation/UAE_RESERVATION_STAT_DAILY_PROCESSED.csv')
#pd.set_option('display.max_columns', None)
#UAE_RESERVATION_STAT_DAILY_PROCESSED_pd.head(10)
#UAE_RESERVATION_STAT_DAILY_PROCESSED_pd.shape

In [0]:
UAE_RESERVATION_STAT_DAILY_PROCESSED_pd.isnull().sum()

In [0]:
#EXCLUSION OF CANCELLED AND NO-SHOW BOOKINGS
UAE_RESERVATION_STAT_DAILY_PROCESSED_2_pd = UAE_RESERVATION_STAT_DAILY_PROCESSED_pd[UAE_RESERVATION_STAT_DAILY_PROCESSED_pd['RESV_STATUS'].isin(['CHECKED IN','CHECKED OUT'])]
print(UAE_RESERVATION_STAT_DAILY_PROCESSED_2_pd.shape)

#EXCLUSION OF FINANCIAL TRANSACTION INTERNAL TO JUMEIRAH
UAE_RESERVATION_STAT_DAILY_PROCESSED_2_pd = UAE_RESERVATION_STAT_DAILY_PROCESSED_2_pd[UAE_RESERVATION_STAT_DAILY_PROCESSED_2_pd['ROOM_CLASS']!='PMA']
UAE_RESERVATION_STAT_DAILY_PROCESSED_2_pd = UAE_RESERVATION_STAT_DAILY_PROCESSED_2_pd[UAE_RESERVATION_STAT_DAILY_PROCESSED_2_pd['MARKET_MAIN_GROUP']!='NON']

print(UAE_RESERVATION_STAT_DAILY_PROCESSED_2_pd.shape)

#EXCLUSION OF RESORT NOT BELONGING TO JUMEIRAH
UAE_RESERVATION_STAT_DAILY_PROCESSED_2_pd = UAE_RESERVATION_STAT_DAILY_PROCESSED_2_pd[UAE_RESERVATION_STAT_DAILY_PROCESSED_2_pd['RESORT']!='ZHA']
UAE_RESERVATION_STAT_DAILY_PROCESSED_2_pd = UAE_RESERVATION_STAT_DAILY_PROCESSED_2_pd[UAE_RESERVATION_STAT_DAILY_PROCESSED_2_pd['RESORT']!='ZMA']
UAE_RESERVATION_STAT_DAILY_PROCESSED_2_pd = UAE_RESERVATION_STAT_DAILY_PROCESSED_2_pd[UAE_RESERVATION_STAT_DAILY_PROCESSED_2_pd['RESORT']!='JAH']
print(UAE_RESERVATION_STAT_DAILY_PROCESSED_2_pd.shape)

#EXCLUSION OF WRONG RESERVATION NAME
UAE_RESERVATION_STAT_DAILY_PROCESSED_2_pd = UAE_RESERVATION_STAT_DAILY_PROCESSED_2_pd[UAE_RESERVATION_STAT_DAILY_PROCESSED_2_pd['RESV_NAME_ID'].astype(int)>0]
print(UAE_RESERVATION_STAT_DAILY_PROCESSED_2_pd.shape)

#EXCLUSION OF DATE NULL
UAE_RESERVATION_STAT_DAILY_PROCESSED_2_pd = UAE_RESERVATION_STAT_DAILY_PROCESSED_2_pd.dropna(subset=['TRUNC_BEGIN_DATE', 'TRUNC_END_DATE'])
print(UAE_RESERVATION_STAT_DAILY_PROCESSED_2_pd.shape)

# BUSINESS LOGIC 1
mask=(UAE_RESERVATION_STAT_DAILY_PROCESSED_2_pd["TRUNC_BEGIN_DATE"]==UAE_RESERVATION_STAT_DAILY_PROCESSED_2_pd["TRUNC_END_DATE"])&(UAE_RESERVATION_STAT_DAILY_PROCESSED_2_pd["TOTAL_REVENUE"].astype(float)>0)
UAE_RESERVATION_STAT_DAILY_PROCESSED_2_pd.loc[mask,["NIGHTS"]]=1

# DROPPING DUPLICATES
UAE_RESERVATION_STAT_DAILY_PROCESSED_2_pd.drop_duplicates(keep='last',inplace=True)
print(UAE_RESERVATION_STAT_DAILY_PROCESSED_2_pd.shape)

# REMOVING BOOKINGS WITH ROOMNIGHT 0
# print('REMOVING BOOKINGS WITH ROOMNIGHT 0')
# bookingswithatlessthanoneroomnight=UAE_RESERVATION_STAT_DAILY_PROCESSED_2_pd[UAE_RESERVATION_STAT_DAILY_PROCESSED_2_pd["PSUEDO_ROOM_YN"]=="N"].groupby(["RESV_NAME_ID"])["NIGHTS"].sum().to_frame()
# print(bookingswithatlessthanoneroomnight[bookingswithatlessthanoneroomnight["NIGHTS"]<1].shape)
# bookingswithatlessthanoneroomnight=bookingswithatlessthanoneroomnight[bookingswithatlessthanoneroomnight["NIGHTS"]<1]
# print(bookingswithatlessthanoneroomnight.shape)


# print(UAE_RESERVATION_STAT_DAILY_PROCESSED_2_pd.shape)
# mask=UAE_RESERVATION_STAT_DAILY_PROCESSED_2_pd["RESV_NAME_ID"].isin(bookingswithatlessthanoneroomnight.index)
# UAE_RESERVATION_STAT_DAILY_PROCESSED_2_pd=UAE_RESERVATION_STAT_DAILY_PROCESSED_2_pd.loc[~mask,:]
# print("REMOVED..FINAL SHAPE")
# print(UAE_RESERVATION_STAT_DAILY_PROCESSED_2_pd.shape)


#SAVING INTO A CLEANSED FILE
UAE_RESERVATION_STAT_DAILY_PROCESSED_2_pd.to_csv('/dbfs/mnt/jumseg/reservation/UAE_RESERVATION_STAT_DAILY_PROCESSED_2_2_RFMV3.csv')
UAE_RESERVATION_STAT_DAILY_PROCESSED_2_pd.shape

In [0]:
df_num_bookings=UAE_RESERVATION_STAT_DAILY_PROCESSED_2_pd[UAE_RESERVATION_STAT_DAILY_PROCESSED_2_pd["PSUEDO_ROOM_YN"]=="N"].groupby(['NAME_ID'])["RESV_NAME_ID"].nunique().to_frame().rename(columns={"RESV_NAME_ID":"NUMBER_BOOKING"})
print(df_num_bookings.shape)

df_num_nights=UAE_RESERVATION_STAT_DAILY_PROCESSED_2_pd[UAE_RESERVATION_STAT_DAILY_PROCESSED_2_pd["PSUEDO_ROOM_YN"]=="N"].groupby(['NAME_ID'])["NIGHTS"].sum().to_frame().rename(columns={"NIGHTS":"NUMBER_NIGHTS"})
print(df_num_nights.shape)

consolidated=df_num_nights.merge(df_num_bookings,how='left',right_index=True,left_index=True)
print(consolidated.shape)

consolidated.head()

In [0]:
import pandas as pd
pd.set_option('display.max_columns', None)
UAE_RESERVATION_STAT_DAILY_PROCESSED_2_pd=pd.read_csv('/dbfs/mnt/jumseg/reservation/UAE_RESERVATION_STAT_DAILY_PROCESSED_2_2_RFMV3.csv')
UAE_RESERVATION_STAT_DAILY_PROCESSED_2_pd.head()

In [0]:
UAE_RESERVATION_STAT_DAILY_PROCESSED_2_pd["BUSINESS_DATE"]=pd.to_datetime(UAE_RESERVATION_STAT_DAILY_PROCESSED_2_pd["BUSINESS_DATE"])
UAE_RESERVATION_STAT_DAILY_PROCESSED_2_pd["TRUNC_BEGIN_DATE"]=pd.to_datetime(UAE_RESERVATION_STAT_DAILY_PROCESSED_2_pd["TRUNC_BEGIN_DATE"])
UAE_RESERVATION_STAT_DAILY_PROCESSED_2_pd["TRUNC_END_DATE"]=pd.to_datetime(UAE_RESERVATION_STAT_DAILY_PROCESSED_2_pd["TRUNC_END_DATE"])
UAE_RESERVATION_STAT_DAILY_PROCESSED_2_pd.sort_values("BUSINESS_DATE",inplace=True)


In [0]:
UAE_RESERVATION_STAT_DAILY_PROCESSED_2_pd["BUSINESS_DATE"].max(),UAE_RESERVATION_STAT_DAILY_PROCESSED_2_pd["BUSINESS_DATE"].min()

In [0]:
UAE_RESERVATION_STAT_DAILY_PROCESSED_2_pd.isnull().sum()

In [0]:
# UAE_RESERVATION_STAT_DAILY_PROCESSED_2_pd["first_checkout"]=UAE_RESERVATION_STAT_DAILY_PROCESSED_2_pd[UAE_RESERVATION_STAT_DAILY_PROCESSED_2_pd["RESV_STATUS"]=="CHECKED OUT"].groupby("NAME_ID")["BUSINESS_DATE"].first()
# UAE_RESERVATION_STAT_DAILY_PROCESSED_2_pd["last_checkout"]=UAE_RESERVATION_STAT_DAILY_PROCESSED_2_pd[UAE_RESERVATION_STAT_DAILY_PROCESSED_2_pd["RESV_STATUS"]=="CHECKED OUT"].groupby("NAME_ID")["BUSINESS_DATE"].last()
# UAE_RESERVATION_STAT_DAILY_PROCESSED_2_pd.head()

In [0]:
from datetime import datetime
datetime(2019,9,1)

In [0]:
name_master=UAE_RESERVATION_STAT_DAILY_PROCESSED_2_pd.groupby("NAME_ID")["RESV_NAME_ID"].nunique().to_frame().rename(columns={"RESV_NAME_ID": "total_bookings"})


tempbis = UAE_RESERVATION_STAT_DAILY_PROCESSED_2_pd.groupby("NAME_ID")["BUSINESS_DATE"].min().to_frame().rename(columns={"BUSINESS_DATE": "first_checkin_date"})
name_master=name_master.merge(tempbis,how='left',right_index=True,left_index=True)

temp=UAE_RESERVATION_STAT_DAILY_PROCESSED_2_pd.groupby(["NAME_ID","RESV_NAME_ID"])["BUSINESS_DATE"].max().to_frame().reset_index()
tempbis = temp.groupby("NAME_ID")["BUSINESS_DATE"].min().to_frame().rename(columns={"BUSINESS_DATE": "first_checkout_date"})
name_master=name_master.merge(tempbis,how='left',right_index=True,left_index=True)

tempbis = temp.groupby("NAME_ID")["BUSINESS_DATE"].max().to_frame().rename(columns={"BUSINESS_DATE": "last_checkout_date"})
name_master=name_master.merge(tempbis,how='left',right_index=True,left_index=True)

tempbis = UAE_RESERVATION_STAT_DAILY_PROCESSED_2_pd.groupby("NAME_ID")["NIGHTS"].sum().to_frame().rename(columns={"NIGHTS": "stay_nights"})
name_master=name_master.merge(tempbis,how='left',right_index=True,left_index=True)



name_master["2+ year old customer"]=name_master["first_checkout_date"].apply(lambda x: 1 if x<datetime(2017,9,1) else 0)

# datetime(2019,1,9).year
def calcYear(x):
  month=x.month
  year=x.year
  if (month>=9):
    return year+1
  else:
    return year
name_master["year_of_first_checkin"]=name_master["first_checkin_date"].apply(calcYear)
UAE_RESERVATION_STAT_DAILY_PROCESSED_2_pd["year"]=UAE_RESERVATION_STAT_DAILY_PROCESSED_2_pd["BUSINESS_DATE"].apply(calcYear)
temp=UAE_RESERVATION_STAT_DAILY_PROCESSED_2_pd.groupby(["NAME_ID","year"])['NIGHTS'].sum().to_frame().unstack()
temp.columns=[ "nights 2017","nights 2018","nights 2019","nights 2020"]
temp.fillna(0,inplace=True)
name_master=name_master.merge(temp,how='left',right_index=True,left_index=True)

temp=UAE_RESERVATION_STAT_DAILY_PROCESSED_2_pd.groupby(["NAME_ID","RESV_NAME_ID"])["BUSINESS_DATE"].min().to_frame().reset_index().rename(columns={"BUSINESS_DATE":"resv_checkin_date"})
temp["checkin_year"]=temp["resv_checkin_date"].apply(calcYear)
temp=temp.groupby(["NAME_ID","checkin_year"])["RESV_NAME_ID"].count().to_frame().unstack()
temp.columns=["2017 bookings","2018 bookings","2019 bookings","2020 bookings"]
temp.fillna(0,inplace=True)
name_master=name_master.merge(temp,how='left',right_index=True,left_index=True)

def year3calc(x):
  if ((x["nights 2017"]!=0) & (x["nights 2018"]!=0) & (x["nights 2019"]!=0) ):
    return 1
  else:
    return 0
name_master["all 3 years stay?"]=name_master.apply(year3calc,axis=1)
def year2calc(x):
  if (x["all 3 years stay?"]==0):
    nightlist=[x["nights 2017"]]
    nightlist.append(x["nights 2018"])
    nightlist.append(x["nights 2019"])
    zeros=[t for t in nightlist if t==0]
    if(len(zeros)==1):
      return 1
    else:
      return 0
  else:
    return 0 
  
name_master["2 years stay?"]=name_master.apply(year2calc,axis=1)

def yearaltcalc(x):
  if (x["2 years stay?"]==1):
    if ((x["nights 2017"]!=0)&(x["nights 2019"]!=0)):
      return 1
    elif ((x["nights 2018"]!=0)&(x["nights 2020"]!=0)):
      return 1
    else:
      return 0
  else:
    return 0
name_master["alternate years stay?"]=name_master.apply(yearaltcalc,axis=1)

name_master["one-timer"]=name_master["total_bookings"].apply(lambda x: 1 if x==1 else 0)

mask=((name_master["nights 2017"]==0) & (name_master["nights 2018"]==0) & (name_master["nights 2019"]!=0))
name_master.loc[mask,"newly-acquired"]=1
name_master.fillna(0,inplace=True)
print(name_master.shape)

name_master=name_master[name_master["year_of_first_checkin"]!=2020]
print(name_master.shape)

mask=((name_master["nights 2017"]!=0) & (name_master["nights 2018"]==0) & (name_master["nights 2019"]==0))
name_master.loc[mask,"node1 lapsed?"]="lapsed"
name_master.loc[~mask,"node1 lapsed?"]="active"

mask= ((name_master["node1 lapsed?"]=="active") & (name_master["all 3 years stay?"]==1))
name_master.loc[mask,"node2.1"]="regular guests"
mask= ((name_master["node1 lapsed?"]=="active") & ~(name_master["all 3 years stay?"]==1))
name_master.loc[mask,"node2.1"]="less freq guests"

mask =(name_master["node1 lapsed?"]=="active")&(name_master["node2.1"]=="less freq guests")&(name_master["newly-acquired"]==1)
name_master.loc[mask,"node3.1"]="newly-acquired"
mask =(name_master["node1 lapsed?"]=="active")&(name_master["node2.1"]=="less freq guests")&(name_master["one-timer"]==1)
name_master.loc[mask,"node3.1"]="one-timer"
mask =(name_master["node1 lapsed?"]=="active")&(name_master["node2.1"]=="less freq guests")&(name_master["alternate years stay?"]==1)
name_master.loc[mask,"node3.1"]="alternate-year"
mask =(name_master["node1 lapsed?"]=="active")&(name_master["node2.1"]=="less freq guests")&(name_master["node3.1"].isnull())
name_master.loc[mask,"node3.1"]="irregulars"

# name_master["node3.1"].fillna("",inplace=True)
mask= ((name_master["node1 lapsed?"]=="active") & (name_master["node2.1"]=="regular guests") & (name_master["2017 bookings"]>=2) & (name_master["2018 bookings"]>=2) & (name_master["2019 bookings"]>=2))
name_master.loc[mask,"node3.1"]="loyals"
mask= ((name_master["node1 lapsed?"]=="active") & (name_master["node2.1"]=="regular guests") & ((name_master["2017 bookings"]<2) | (name_master["2018 bookings"]<2) | (name_master["2019 bookings"]<2)))
name_master.loc[mask,"node3.1"]="regulars"


name_master["Recency"]=name_master["last_checkout_date"].apply(lambda x: (datetime(2019,9,1)-x).days)

name_master.head()

In [0]:
def calcfreq(x):
  try:
    freq= x["stay_nights"]/ (2019-x["year_of_first_checkin"]+1)
    return freq
  except:
    pass
  
name_master["Frequency (stay nights per year)"]=name_master.apply(calcfreq,axis=1)

def calcfreq2(x):
  try:
    freq= x["total_bookings"]/ (2019-x["year_of_first_checkin"]+1)
    return freq
  except:
    pass
  
name_master["Frequency2 (bookings per year)"]=name_master.apply(calcfreq2,axis=1)

def calcfreq3(x):
  try:
    freq= x["total_bookings"]/ (3)
    return freq
  except:
    pass
  
name_master["Frequency (number of bookings in 3 years)"]=name_master.apply(calcfreq3,axis=1)

temp=UAE_RESERVATION_STAT_DAILY_PROCESSED_2_pd.groupby(["NAME_ID"])["TOTAL_REVENUE"].sum().to_frame()
name_master=name_master.merge(temp,how='left',right_index=True,left_index=True).rename(columns={"TOTAL_REVENUE":"Monetary Total"})

name_master["Spend per night"]=name_master["Monetary Total"]/name_master["stay_nights"]

temp=UAE_RESERVATION_STAT_DAILY_PROCESSED_2_pd.groupby(["NAME_ID"])["ROOM_REVENUE"].sum().to_frame()
name_master=name_master.merge(temp,how='left',right_index=True,left_index=True).rename(columns={"ROOM_REVENUE":"Monetary Room"})

temp=UAE_RESERVATION_STAT_DAILY_PROCESSED_2_pd.groupby(["NAME_ID"])["FOOD_REVENUE"].sum().to_frame()
name_master=name_master.merge(temp,how='left',right_index=True,left_index=True).rename(columns={"FOOD_REVENUE":"Monetary Food"})
# TODO: CHECK WITH ABD
name_master=name_master[name_master["Monetary Room"]>0]

# TODO: CHECK WITH ABD
name_master=name_master[name_master["stay_nights"]>0]

name_master.head()

In [0]:
name_master.groupby(["node1 lapsed?"])["total_bookings"].count().to_frame()

In [0]:
(name_master.fillna("lapsed").groupby(["node1 lapsed?","node2.1","node3.1"])["total_bookings"].count()).to_frame()

In [0]:
name_master.fillna("lapsed").groupby(["node1 lapsed?","node2.1","node3.1"])["Monetary Total"].sum().to_frame()

In [0]:
name_master.to_csv("/dbfs/FileStore/tables/name_master.csv")

In [0]:
name_master["Spend per night"].describe()

In [0]:

name_master.shape

In [0]:
# name_master["Spend per night"]
name_master[name_master.index==73246939.0]
UAE_RESERVATION_STAT_DAILY_PROCESSED_2_pd[UAE_RESERVATION_STAT_DAILY_PROCESSED_2_pd["NAME_ID"]==73246939]
print(name_master.shape)
name_master[name_master["stay_nights"]==0].head()

In [0]:

UAE_RESERVATION_STAT_DAILY_PROCESSED_2_pd[UAE_RESERVATION_STAT_DAILY_PROCESSED_2_pd["NAME_ID"]==410533.0]

In [0]:
# name_master[name_master["node3.1"].isnull()].sample(10)

In [0]:
# def node3(x):
#   if ((x["node1 lapsed?"]=="active") & (x["node2.1"]=="regular guests") & (x) & (x))

In [0]:
# name_master.columns

In [0]:
# name_master=name_master[['total_bookings','stay_nights', 'nights 2017', 'nights 2018', 'nights 2019',
#        'nights 2020', '2017 bookings', '2018 bookings', '2019 bookings',
#        '2020 bookings', 'all 3 years stay?', '2 years stay?',
#        'alternate years stay?', 'one-timer', 'newly-acquired', 'node1 lapsed?',
#        'node2.1', 'node3.1']]

In [0]:
# # temp=UAE_RESERVATION_STAT_DAILY_PROCESSED_2_pd.groupby(["NAME_ID","RESV_NAME_ID"])["BUSINESS_DATE"].max().to_frame().reset_index()
# # tempbis = temp.groupby("NAME_ID")["BUSINESS_DATE"].min().to_frame().rename(columns={"BUSINESS_DATE": "first_checkout_date"})
# temp=UAE_RESERVATION_STAT_DAILY_PROCESSED_2_pd.groupby(["NAME_ID","RESV_NAME_ID"])["BUSINESS_DATE"].min().to_frame().reset_index().rename(columns={"BUSINESS_DATE":"resv_checkin_date"})
# temp["checkin_year"]=temp["resv_checkin_date"].apply(calcYear)
# temp=temp.groupby(["NAME_ID","checkin_year"])["RESV_NAME_ID"].count().to_frame().unstack()
# temp.columns=["2017 bookings","2018 bookings","2019 bookings","2020 bookings"]
# temp.fillna(0,inplace=True)
# temp.head()


In [0]:
# UAE_RESERVATION_STAT_DAILY_PROCESSED_2_pd["year"]=UAE_RESERVATION_STAT_DAILY_PROCESSED_2_pd["BUSINESS_DATE"].apply(calcYear)
# UAE_RESERVATION_STAT_DAILY_PROCESSED_2_pd[["BUSINESS_DATE","year"]].head(10)

# temp.head()
# name_master["all 3 years stay?"].value_counts()
# name_master["2 years stay?"].value_counts()
# name_master[name_master["2 years stay?"]==1].head()
# name_master["alternate years stay?"].value_counts()
# name_master[name_master["alternate years stay?"]==1].head()

# name_master["newly-acquired"].value_counts()
# name_master[name_master["newly-acquired"]==1].head()
# name_master["nights 2020"].value_counts()
# name_master[name_master["nights 2020"]==1]
# name_master["node1 lapsed?"].value_counts(dropna=False)
# name_master["node2.1"].value_counts(dropna=False)
# name_master[name_master["node2.1"]=="less freq guests"]
# name_master["node3.1"].value_counts(dropna=False)
name_master["node3.1"].value_counts(dropna=False)

In [0]:
name_master["node1 lapsed?"].value_counts(dropna=False)

In [0]:
name_master["2+ year old customer"].value_counts()

In [0]:
# name_master["<1 year (new)customer"]=name_master["first_checkout_date"].apply(lambda x: 1 if x>datetime(2018,9,1) else 0)
name_master.head()

In [0]:
# name_master["<1 year (new)customer"].value_counts(dropna=False)

In [0]:
name_master["Recency"]=name_master["last_checkout_date"].apply(lambda x: (datetime(2019,9,1)-x).days)
name_master.head()

In [0]:
name_master["year_of_first_checkin"].value_counts()

In [0]:

name_master.head()

In [0]:

name_master.head()

In [0]:
segmentablecustomers=name_master[name_master["Recency"]>=0]
# segmentablecustomers=segmentablecustomers[segmentablecustomers["2+ year old customer"]==1]
# segmentablecustomers=segmentablecustomers[segmentablecustomers["<1 year (new)customer"]==0]
# segmentablecustomers=segmentablecustomers[segmentablecustomers["total_bookings"]>1]
segmentablecustomers.dropna(inplace=True)
name_master.shape[0],segmentablecustomers.shape[0]

In [0]:
segmentablecustomers.isnull().sum()

In [0]:
name_master[name_master["year_of_first_checkin"]==2019].head()

In [0]:
UAE_RESERVATION_STAT_DAILY_PROCESSED_2_pd[UAE_RESERVATION_STAT_DAILY_PROCESSED_2_pd["NAME_ID"]==378492].sort_values("RESV_NAME_ID")

In [0]:
segmentablecustomers[segmentablecustomers["Frequency"]==1].head()

In [0]:
# name_master.to_csv("/dbfs/FileStore/tables/name_master.csv")
segmentablecustomers.to_csv("/dbfs/FileStore/tables/segmentable_master.csv")

In [0]:
# import spark_df_profiling
file="FileStore/tables/segmentable_master.csv"
tempdf = (spark.read
    .format('com.databricks.spark.csv')
    .options(header=True)
    .load(file))

profile = spark_df_profiling.ProfileReport(tempdf)
profile.to_file(outputfile="/dbfs/FileStore/tables/Psegmentablemaster.html")


In [0]:
UAE_RESERVATION_STAT_DAILY_PROCESSED_2_pd.head()

In [0]:
name_master["Recency"].describe()

In [0]:
name_master[name_master["Recency"]<0].count()

In [0]:
name_master.dtypes

In [0]:
name_master.isnull().sum()

In [0]:
name_master[name_master["2+ year old customer"]==0].head()

In [0]:
pd.set_option('display.max_columns', None)
379328.0
UAE_RESERVATION_STAT_DAILY_PROCESSED_2_pd[UAE_RESERVATION_STAT_DAILY_PROCESSED_2_pd["NAME_ID"]==379328]

In [0]:
df_num_bookings=UAE_RESERVATION_STAT_DAILY_PROCESSED_2_pd[UAE_RESERVATION_STAT_DAILY_PROCESSED_2_pd["PSUEDO_ROOM_YN"]=="N"].groupby(['NAME_ID'])["RESV_NAME_ID"].nunique().to_frame().rename(columns={"RESV_NAME_ID":"NUMBER_BOOKING"})
print(df_num_bookings.shape)
df_num_bookings.head(10)

In [0]:
df_num_bookings.sort_values("NUMBER_BOOKING",ascending=False).head(10)

In [0]:
Number_Bookings = UAE_RESERVATION_STAT_DAILY_PROCESSED_2_pd.groupby(['NAME_ID'])['RESV_NAME_ID'].nunique().to_frame()
Number_Bookings.columns = ['NUMBER_BOOKING']
print(Number_Bookings.shape)
Number_Bookings.head(10)

In [0]:
Number_Bookings.sort_values("NUMBER_BOOKING",ascending=False).head(10)

In [0]:
df_num_nights=UAE_RESERVATION_STAT_DAILY_PROCESSED_2_pd[UAE_RESERVATION_STAT_DAILY_PROCESSED_2_pd["PSUEDO_ROOM_YN"]=="N"].groupby(['NAME_ID'])["NIGHTS"].sum().to_frame().rename(columns={"NIGHTS":"NUMBER_NIGHTS"})
print(df_num_nights.shape)
df_num_nights.head()

In [0]:
Number_Nights = UAE_RESERVATION_STAT_DAILY_PROCESSED_2_pd.groupby(['NAME_ID'])['NIGHT_BIS'].sum().to_frame()
Number_Nights.columns = ['NUMBER_NIGHTS']
print(Number_Nights.shape)
Number_Nights.head(10)

In [0]:
testdf=Number_Nights.merge(df_num_nights,how='left',right_index=True,left_index=True).dropna()
testdf[testdf["NUMBER_NIGHTS_x"]!=testdf["NUMBER_NIGHTS_y"]].head(10)

In [0]:
UAE_RESERVATION_STAT_DAILY_PROCESSED_2_pd[UAE_RESERVATION_STAT_DAILY_PROCESSED_2_pd['NAME_ID']==383751].sort_values(['RESV_NAME_ID'])
# shared?

In [0]:
# wHY NIGHTS IS SHOWING 0
UAE_RESERVATION_STAT_DAILY_PROCESSED_2_pd[UAE_RESERVATION_STAT_DAILY_PROCESSED_2_pd['NAME_ID']==396284].sort_values(['RESV_NAME_ID'])

In [0]:
UAE_RESERVATION_STAT_DAILY_PROCESSED_2_pd[UAE_RESERVATION_STAT_DAILY_PROCESSED_2_pd['NAME_ID']==14994].head()

In [0]:
UAE_RESERVATION_STAT_DAILY_PROCESSED_2_pd[UAE_RESERVATION_STAT_DAILY_PROCESSED_2_pd['NAME_ID']==377142.0]

In [0]:
386948.0
UAE_RESERVATION_STAT_DAILY_PROCESSED_2_pd[UAE_RESERVATION_STAT_DAILY_PROCESSED_2_pd['NAME_ID']==386948.0]

In [0]:
# UAE_RESERVATION_STAT_DAILY_PROCESSED_2_pd.to_csv()

In [0]:
UAE_RESERVATION_STAT_DAILY_PROCESSED_2_pd[(UAE_RESERVATION_STAT_DAILY_PROCESSED_2_pd['PSUEDO_ROOM_YN']=='Y')]["MARKET_MAIN_GROUP"].value_counts()

In [0]:
UAE_RESERVATION_STAT_DAILY_PROCESSED_2_pd[(UAE_RESERVATION_STAT_DAILY_PROCESSED_2_pd['PSUEDO_ROOM_YN']=='Y')]["MARKET_CODE"].value_counts()

In [0]:
UAE_RESERVATION_STAT_DAILY_PROCESSED_2_pd[["MARKET_CODE","MARKET_MAIN_GROUP"]].drop_duplicates(keep='first').sort_values(["MARKET_MAIN_GROUP","MARKET_CODE"])

In [0]:
# UAE_RESERVATION_STAT_DAILY_PROCESSED_2_pd[].groupby(["MARKET_MAIN_GROUP","ROOM_CLASS"])["MARKET_CODE"].count()

In [0]:
UAE_RESERVATION_STAT_DAILY_PROCESSED_2_pd[UAE_RESERVATION_STAT_DAILY_PROCESSED_2_pd["NAME_ID"]==72198519]

In [0]:
UAE_RESERVATION_STAT_DAILY_PROCESSED_2_pd[UAE_RESERVATION_STAT_DAILY_PROCESSED_2_pd["NAME_ID"]==71510600.0]

In [0]:
# UAE_RESERVATION_STAT_DAILY_PROCESSED_2_pd["TRUNC_BEGIN_DATE"]=pd.to_datetime(UAE_RESERVATION_STAT_DAILY_PROCESSED_2_pd["TRUNC_BEGIN_DATE"])
# UAE_RESERVATION_STAT_DAILY_PROCESSED_2_pd["TRUNC_END_DATE"]=pd.to_datetime(UAE_RESERVATION_STAT_DAILY_PROCESSED_2_pd["TRUNC_END_DATE"])
UAE_RESERVATION_STAT_DAILY_PROCESSED_2_pd[UAE_RESERVATION_STAT_DAILY_PROCESSED_2_pd["TRUNC_BEGIN_DATE"]==UAE_RESERVATION_STAT_DAILY_PROCESSED_2_pd["TRUNC_END_DATE"]].sort_values("RESV_NAME_ID")["RESV_STATUS"].value_counts()

In [0]:
# UAE_RESERVATION_STAT_DAILY_PROCESSED_2_pd[(UAE_RESERVATION_STAT_DAILY_PROCESSED_2_pd['PSUEDO_ROOM_YN']=='Y')].groupby(["NAME_ID"])
# UAE_RESERVATION_STAT_DAILY_PROCESSED_2_pd.
consolidated=df_num_nights.merge(df_num_bookings,how='left',right_index=True,left_index=True)
# 73440107 dOESNT APPEAR IN OCIS
# 3078611 OCIS ID

consolidated[consolidated["NUMBER_NIGHTS"]==0][["NUMBER_BOOKING","NUMBER_NIGHTS"]].sort_values("NUMBER_BOOKING",ascending=False)

In [0]:
802307
3078611
consolidated[consolidated["NUMBER_BOOKING"]>consolidated["NUMBER_NIGHTS"]][["NUMBER_BOOKING","NUMBER_NIGHTS"]].sort_values("NUMBER_BOOKING",ascending=False)

In [0]:
UAE_RESERVATION_STAT_DAILY_PROCESSED_2_pd.groupby(["RESV_NAME_ID"])["NAME_ID"].nunique().to_frame().sort_values("NAME_ID",ascending=False).head(10)

In [0]:
UAE_RESERVATION_STAT_DAILY_PROCESSED_2_pd[UAE_RESERVATION_STAT_DAILY_PROCESSED_2_pd["RESV_NAME_ID"]==85088105]

In [0]:
import pandas as pd
pd.set_option('display.max_columns', None)
#UAE_RESERVATION_STAT_DAILY_PROCESSED_2_pd[(UAE_RESERVATION_STAT_DAILY_PROCESSED_2_pd['RESV_NAME_ID']==84018413)].head()
UAE_RESERVATION_STAT_DAILY_PROCESSED_2_pd[(UAE_RESERVATION_STAT_DAILY_PROCESSED_2_pd['PSUEDO_ROOM_YN']=='Y')].head()
#UAE_RESERVATION_STAT_DAILY_PROCESSED_2_pd[(UAE_RESERVATION_STAT_DAILY_PROCESSED_2_pd['RESV_NAME_ID']==83753730)].head(20)
#UAE_RESERVATION_STAT_DAILY_PROCESSED_2_pd.REGION_CODE.unique()

In [0]:
UAE_RESERVATION_STAT_DAILY_PROCESSED_pd_filter = UAE_RESERVATION_STAT_DAILY_PROCESSED_pd_filter.groupby(['RESORT','SOURCE_CODE','MARKET_MAIN_GROUP','MARKET_CODE','CHANNEL','WALKIN_YN','RESERVATION_DATE','PSUEDO_ROOM_YN','ROOM_CLASS','ROOM_CATEGORY','RESV_NAME_ID','RESV_STATUS','CANCELLATION_DATE','TRUNC_BEGIN_DATE','TRUNC_END_DATE','NIGHTS','NAME_ID','ADULTS','CHILDREN','VIP_STATUS','BIRTH_DATE','MEMBERSHIP_ID','COUNTRY','REGION_CODE','NATIONALITY'])['ROOM_REVENUE','FOOD_REVENUE','TOTAL_REVENUE','PACKAGE_ROOM_REVENUE','PACKAGE_FOOD_REVENUE'].sum()
UAE_RESERVATION_STAT_DAILY_PROCESSED_pd_filter.shape

In [0]:
UAE_RESERVATION_STAT_DAILY_PROCESSED_pd_filter.head()

In [0]:
bookings.registerTempTable("bookings_temp")
%sql
select * from bookings_temp where address_id = '14568225'
bookings_bis =bookings_temp.filter(col("NAME_ID').isin("929041"))
bookings_bis[bookings_bis['NAME_ID']=='929041']
bookings.filter(bookings("TRUNC_ACTUAL_CHECK_IN_DATE") >= bookings.unix_timestamp(f.lit('2016-09-01 00:00:00')).cast('timestamp'))
bookings.where(bookings.TRUNC_ACTUAL_CHECK_IN_DATE.cast('timestamp') >= date('2016-09-01 00:00:00
bookings.registerTempTable("temp")
%sql
select count(*) from temp where cast(TRUNC_ACTUAL_CHECK_IN_DATE as DATE) >= '2016-09-01'
filter_df = spark.sql("select * from temp where cast(TRUNC_ACTUAL_CHECK_IN_DATE as DATE) >= cast('2016-09-01' as date)")
filter_df.count()
filter_df.select('RESORT','RESV_NAME_ID','RESV_STATUS','CONFIRMATION_NO','BEGIN_DATE','END_DATE','PARENT_RESV_NAME_ID','WALKIN_YN','MEMBERSHIP_ID','CHANNEL').coalesce(1).write.format("com.databricks.spark.csv").options(header=True).save("dbfs:/mnt/jumseg/reservation/UAE_pms_reservation_name_filter.csv")